In [2]:
import pandas as pd
import numpy as np 
import os, sys
from stylegan_layers import  G_mapping,G_synthesis
from sklearn.neighbors import KernelDensity
import shutil
from torchvision.utils import save_image
import torch
#from stylegan_layers import  G_mapping,G_synthesis, Truncation
import pickle
import matplotlib.patches as patches
from collections import OrderedDict
import torch.nn.functional as F
from sklearn.neighbors import KernelDensity
from torchvision.utils import save_image
from typing import final
from tqdm import tqdm
from PIL import Image, ImageOps
from torchvision import transforms
import torchvision
import matplotlib.image as mpimg
from torchvision import models
import cv2
import torch.nn as nn
import pandas as pd
from torchvision.utils import save_image
import pydicom as dicom
import numpy as np
from math import log10
import matplotlib.pyplot as plt
from model import * 
from utils import * 
import imageio
sys.path.append("/home/jessica/labelGAN/datasetGAN_release/models")
from stylegan1 import G_mapping,Truncation,G_synthesis


In [3]:
device = 'cuda:3' if torch.cuda.is_available() else 'cpu'
avg_latent = np.load('/data3/jessica/data/labelGAN//generations/avg_latent_stylegan1.npy')
avg_latent = torch.from_numpy(avg_latent).type(torch.FloatTensor).to(device)
g_all = nn.Sequential(OrderedDict([('g_mapping', G_mapping()),
    ('truncation', Truncation(avg_latent,max_layer=8, threshold=0.7, device=device)),
    ('g_synthesis', G_synthesis(resolution=1024))    
    ]))

#opts = {'mapping_layers': 8, 'truncation_psi': -1.}
'''g_all = Generator(resolution=1024,
                    num_channels=3,
                    structure='linear',
                    **opts)'''
'''g_all = nn.Sequential(OrderedDict([('g_mapping', GMapping()),
    ('truncation', Truncation(avg_latent)),
    ('g_synthesis', GSynthesis(resolution=1024, depth=8))    
    ]))'''
#Load the pre-trained model
ckpt = torch.load('/data1/jessica/data/labelGAN/checkpoints/styleGAN/GAN_GEN_8.pth', map_location=device)
for key in list(ckpt.keys()):
            new_key = key.replace('init_block', 'blocks.4x4').replace('blocks.0.', 'blocks.8x8.')
            new_key = new_key.replace('blocks.1.', 'blocks.16x16.').replace('blocks.5.', 'blocks.256x256.')
            new_key = new_key.replace('blocks.3.', 'blocks.64x64.').replace('blocks.2.', 'blocks.32x32.')
            new_key = new_key.replace('blocks.4.', 'blocks.128x128.').replace('blocks.6.', 'blocks.512x512.')

            new_key = new_key.replace('blocks.7.', 'blocks.1024x1024.')
            
            new_key = new_key.replace("g_mapping.map.dense", "g_mapping.dense")
            new_key = new_key.replace("g_synthesis.to_rgb.8.", "g_synthesis.torgb.")
            
            ckpt[new_key] = ckpt.pop(key)

g_all.load_state_dict(ckpt, strict=False)
g_all.eval()
g_all = g_all.to(device)
for param in g_all.parameters():
    param.requires_grad = False

g_mapping, g_synthesis = g_all.g_mapping, g_all.g_synthesis
g_mapping: final = g_mapping
g_synthesis: final =  g_synthesis
truncation = g_all.truncation.to(device)
perceptual = VGG16_perceptual().to(device)


/home/jessica/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jessica/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
def get_sampls(kde):
    # Generate a set of points at which to evaluate the PDF
    x_grid = np.linspace(-3, 3, 1000)

    # Compute the estimated PDF at the evaluation points
    pdf_vals = np.exp(kde.score_samples(x_grid.reshape(-1, 1)))

    # Compute the CDF of the estimated PDF
    cdf_vals = np.cumsum(pdf_vals) / np.sum(pdf_vals)

    # Generate 100 new samples using inverse transform sampling
    u = np.random.uniform(size=100)
    new_samples = np.interp(u, cdf_vals, x_grid)
    return new_samples

In [5]:
latent_path = '/data3/jessica/data/labelGAN/train_images/latents'

In [8]:
save_path = '/data3/jessica/data/labelGAN/vinbig_sampled_KDE'

In [11]:
image_latents_names = [elem.replace("latent_", "").replace(".npy", "") for elem in os.listdir(latent_path)]
df = pd.read_csv("/data3/jessica/data/labelGAN/vinbig/train.csv")
df_sampled = df[df.image_id.isin(image_latents_names)]
kernel = 'tophat'
for class_name in df.class_name.unique():
   print(class_name)
   test = df_sampled[df_sampled.class_name == class_name].image_id
   class_name = class_name.replace("/", "").replace(" ", "")
   images = test.apply(lambda x: np.load(f"{latent_path}/latent_" + x + ".npy"))
   if len(images) == 0: continue
   t = np.concatenate(np.array(images), axis=0)
   t = t.reshape(t.shape[0], -1)# create an array of samples
   samples = np.array(t)

   # fit a kernel density estimator to the samples
   kde = KernelDensity(bandwidth=0.5, kernel=kernel)
   kde.fit(samples)

   # sample from the kde
   new_samples = kde.sample(200)
   for i, latents in enumerate(new_samples):
      latents = torch.tensor(latents.reshape(1, 18, 512)).to("cuda:3").float()
      np.save(f"{save_path}/{class_name}_{i}_{kernel}.npy", latents.cpu())
      syn_img, _ = g_synthesis(latents)
      syn_img = (syn_img+1.0)/2.0
      save_image(syn_img.clamp(0,1),f"{save_path}/{class_name}_{i}_{kernel}.png")

No finding
Cardiomegaly
Aortic enlargement
Pleural thickening
ILD
Nodule/Mass
Pulmonary fibrosis
Lung Opacity
Atelectasis
Other lesion
Infiltration
Pleural effusion
Calcification
Consolidation
Pneumothorax


: 

In [7]:
df.image_id == image_id

,Unnamed: 0,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
1,4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN
2,33,42d472bdda3ad93dac63c8e5e29977bb,No finding,14,R9,NaN,NaN,NaN,NaN
3,34,2b1293d9c276e5439e499f58ce2e31ab,No finding,14,R8,NaN,NaN,NaN,NaN
4,65,2561ba15f063a8e3a0322ad943738bb5,No finding,14,R2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
370,14749,31ffa866e091965da25a408b4cc7d0db,Pneumothorax,12,R9,1212.0,355.0,1868.0,1192.0
371,15725,0f45742c4d100eeee221f8853d79c9d4,Pneumothorax,12,R10,308.0,1511.0,430.0,1669.0
372,15862,3c63e58fcda26e02fdd6619515399985,Pneumothorax,12,R9,226.0,340.0,950.0,1327.0
373,16769,393909c0d12d67fd08703d36970014e8,Pneumothorax,12,R10,1817.0,515.0,2249.0,1485.0


In [24]:
latents = []
p = '/home/jessica/labelGAN/Image2StyleGAN/images/generated_latents_from_class_distr/'
for i, elem in enumerate(os.listdir(p)):
    latents.append(np.load(p + elem ))
    
    

In [27]:
len(os.listdir(p))

1804

In [26]:
np.array(latents).shape

(1804, 1, 18, 512)

In [22]:
path = '/home/jessica/labelGAN/Image2StyleGAN/images/generated_latents_from_class_distr/Atelectasis_1_tophat.npy'
a = np.load(path)

In [23]:
a.shape

(1, 18, 512)

In [10]:
import os

In [11]:
len(os.listdir('/data3/jessica/data/labelGAN/train_images/latents'))

76